In [34]:
import os
import re
import json
import pandas as pd
import numpy as np
import lyricsgenius
from dotenv import load_dotenv
import openai

load_dotenv('/Users/yushunli/env.')
genius_key = os.getenv('GENIUS_API')
openai.api_key = os.getenv('OPENAI_API_KEY')

from requests.exceptions import HTTPError, Timeout
from lyricsgenius import Genius

genius = lyricsgenius.Genius(genius_key)
#Turn off warning messages from pandas
pd.options.mode.chained_assignment = None  # default='warn'

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import plotly.express as px

In [3]:
genius = lyricsgenius.Genius(genius_key)
genius.verbose = False # Turn off status messages
genius.remove_section_headers = False # Remove section headers (e.g. [Chorus]) from lyrics when searching
genius.skip_non_songs = True # Include hits thought to be non-songs (e.g. track lists)
genius.excluded_terms = ["(Live)"] # Exclude songs with these words in their title

In [4]:
def clean_lyrics(lyrics):
    # Remove all characters before the first "["
    lyrics = re.sub(r'^.*?\[', '[', lyrics)
    
    # Remove text starting from a number followed by "Embed" to the end
    lyrics = re.sub(r'\d+Embed$', '', lyrics)
    
    # Remove lines starting with "See " and ending with "You might also like"
    lyrics = re.sub(r'See .*? You might also like', '', lyrics)
    
    return lyrics

In [85]:
#Create embeddings for each song
def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

# Ingest Discography

In [28]:
artist = "Drake"
albums = ["Room for Improvement", "Comeback Season",
          "So Far Gone", "Thank Me Later", "Take Care", "Nothing Was the Same", "If You're Reading This It's Too Late", 
          #"What a Time to Be Alive", 
          "Views", "More Life", "Scorpion", "Care Package",
          "Dark Lane Demo Tapes", "Scary Hours 2", "Certified Lover Boy", "Honestly, Nevermind", 
          #"Her Loss", #"For All the Dogs"
          ]

In [22]:
album_list = []
for album in albums:
    album = genius.search_album(album, artist)
    album_list.append(album)
    print("Completed Saving Album:", album)

Searching for "Room for Improvement" by Drake...
Completed Saving Album: Album(id, artist, ...)
Searching for "Comeback Season" by Drake...
Completed Saving Album: Album(id, artist, ...)
Searching for "So Far Gone" by Drake...
Completed Saving Album: Album(id, artist, ...)
Searching for "Thank Me Later" by Drake...
Completed Saving Album: Album(id, artist, ...)
Searching for "Take Care" by Drake...
Completed Saving Album: Album(id, artist, ...)
Searching for "Nothing Was the Same" by Drake...


Timeout: Request timed out:
HTTPSConnectionPool(host='genius.com', port=443): Read timed out. (read timeout=5)

In [27]:
album_list2 = []
#For loop through albums but skip the first 5 items in the list
for album in albums[5:]:
    album_save = genius.search_album(album, artist)
    album_list2.append(album_save)
    print("Completed Saving Album:", album)

Searching for "Nothing Was the Same" by Drake...
Completed Saving Album: Nothing Was the Same
Searching for "If You're Reading This It's Too Late" by Drake...
Completed Saving Album: If You're Reading This It's Too Late
Searching for "What a Time to Be Alive" by Drake...


Timeout: Request timed out:
HTTPSConnectionPool(host='genius.com', port=443): Read timed out. (read timeout=5)

In [33]:
album_list3 = []
# For loop through albums but skip the first 5 items in the list
for album in albums[7:]:
    album_save = genius.search_album(album, artist)
    album_list3.append(album_save)
    print("Completed Saving Album:", album)

Searching for "Views" by Drake...
Completed Saving Album: Views
Searching for "More Life" by Drake...
Completed Saving Album: More Life
Searching for "Scorpion" by Drake...
Completed Saving Album: Scorpion
Searching for "Care Package" by Drake...
Completed Saving Album: Care Package
Searching for "Dark Lane Demo Tapes" by Drake...
Completed Saving Album: Dark Lane Demo Tapes
Searching for "Scary Hours 2" by Drake...
Completed Saving Album: Scary Hours 2
Searching for "Certified Lover Boy" by Drake...
Completed Saving Album: Certified Lover Boy
Searching for "Honestly, Nevermind" by Drake...
Completed Saving Album: Honestly, Nevermind


In [34]:
drake_albums = []

for album in album_list:
    drake_albums.append(album)

for album in album_list2:
    drake_albums.append(album)

for album in album_list3:
    drake_albums.append(album)

In [35]:
len(drake_albums)

15

In [36]:
#For loop through drake albums and the list album 
for album in drake_albums:
    album.save_lyrics()

Wrote Lyrics_RoomforImprovement.json.
Wrote Lyrics_ComebackSeason.json.
Wrote Lyrics_SoFarGone.json.
Wrote Lyrics_ThankMeLater.json.
Wrote Lyrics_TakeCare.json.
Wrote Lyrics_NothingWastheSame.json.
Wrote Lyrics_IfYoureReadingThisItsTooLate.json.
Wrote Lyrics_Views.json.
Wrote Lyrics_MoreLife.json.
Wrote Lyrics_Scorpion.json.
Wrote Lyrics_CarePackage.json.
Wrote Lyrics_DarkLaneDemoTapes.json.
Wrote Lyrics_ScaryHours2.json.
Wrote Lyrics_CertifiedLoverBoy.json.
Wrote Lyrics_HonestlyNevermind.json.


# Embed Albums

In [5]:
import os
import pandas as pd
import json

# Create an empty list to store dataframes
dfs = []

# Specify the folder path where JSON files are located
folder_path = 'Drake_Export'

# Iterate through all JSON files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        # Load the JSON file
        with open(os.path.join(folder_path, filename)) as f:
            data = json.load(f)
        
        # Create a dataframe from the 'tracks' dictionary
        df = pd.DataFrame(data['tracks'])
        
        # Break down the 'song' column into separate columns
        df = pd.concat([df.drop(['song'], axis=1), df['song'].apply(pd.Series)], axis=1)
        
        # Add the 'album' column with the album name
        df['album'] = data['name']

        #Add Year column
        df['year'] = data['release_date_components']['year']

        #Add Cover Art column
        #df['cover_art_url'] = data['tracks'][0]['song']['song_art_image_thumbnail_url']
        
        # Append the dataframe to the list
        dfs.append(df)

# Concatenate all dataframes into one
combined_df = pd.concat(dfs, ignore_index=True)

drake_df = combined_df[['album', 'year', 'title', 'lyrics', 'song_art_image_thumbnail_url']]
drake_df['lyrics'] = drake_df['lyrics'].apply(clean_lyrics)
drake_df['lyrics'] = drake_df['lyrics'].apply(lambda x: re.sub(r'\[.*?\]', '', x))
drake_df['lyrics'] = drake_df['lyrics'].str.lstrip('\n')
drake_df['lyrics'] = drake_df['lyrics'].str.lstrip('\n')
#Run Embeddings
#drake_df['embedding'] = drake_df['lyrics'].apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))
#Save the dataframe as a parquet file
#drake_df.to_parquet('drake_album_embeddings.parquet')
#Filter for rows where the length of the lyrics is greater than 0
drake_df = drake_df[drake_df['lyrics'].str.len() > 0]
drake_df.sample(5)

,album,year,title,lyrics,song_art_image_thumbnail_url
38,More Life,2017,Skepta Interlude,"It's a madness\nCheck my account, it's a madne...",https://images.genius.com/9515dd6e99fb20b2945b...
147,Views,2016,Pop Style,"Yeah, yeah\n\n\nDropped outta school now we du...",https://images.genius.com/70445cae0ca33c5fcba1...
182,"Honestly, Nevermind",2022,Massive,I've been alone in my thoughts\nCan't fill thi...,https://images.genius.com/19a07ed74c6341946583...
197,Dark Lane Demo Tapes,2020,D4L,"(808 my crew)\n(Ayy, Southside, where you get ...",https://images.genius.com/1b0d68a486d05a162446...
157,Take Care,2011,Take Care,I know you've been hurt by someone else\nI can...,https://images.rapgenius.com/115f4f1b48a5d9e7b...


In [6]:
#Load parquet file as a dataframe
embeddings = pd.read_parquet('drake_album_embeddings.parquet')
#Join the embedding column to the drake_df dataframe
drake_df = drake_df.merge(embeddings, on=['title', 'album', 'year'], how='left').drop(columns=['lyrics_y']).rename(columns={'lyrics_x': 'lyrics'})

In [135]:
drake_df.album.unique()

array(['Comeback Season', 'Scary Hours 2', 'More Life',
       'For All The Dogs', 'So Far Gone', 'Thank Me Later', 'Scorpion',
       'Views', 'Take Care', 'Honestly, Nevermind',
       'Dark Lane Demo Tapes', 'Certified Lover Boy', 'Care Package',
       'Room for Improvement', 'Nothing Was the Same',
       'If You’re Reading This It’s Too Late'], dtype=object)

### Aggregating Embedding By Album

#### Mean Approach

In [8]:
grouped = drake_df.groupby('album')

avg_embeddings = {}

for name, group in grouped:
    # Stack embeddings to form a 2D NumPy array
    embeddings_matrix = np.vstack(group['embedding'].values)
    
    # Calculate the mean along axis 0 (i.e., column-wise mean)
    avg_embedding = np.mean(embeddings_matrix, axis=0)
    
    # Store the result in the dictionary
    avg_embeddings[name] = avg_embedding

In [9]:
album_mean = pd.DataFrame.from_dict(avg_embeddings, orient='index')
# Optionally, reset the index so that album names become a column
#album_avg.reset_index(inplace=True)
#album_avg.columns = ['album_name'] + [f'embedding_dim_{i}' for i in range(album_avg.shape[1]-1)]
#Remove the rows where index is 'Room for Improvement' and 'Scary Hours 2'
#album_avg = album_avg.drop(['Room for Improvement', 'Scary Hours 2'])
album_mean.sample(5)

,0,1,2,3,4,5,6,7,8,9,...,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535
Nothing Was the Same,-0.005901,-0.011415,0.007735,-0.016328,-0.008330,0.014566,0.015501,-0.014117,-0.027435,-0.020751,...,0.018036,-0.005064,-0.001589,-0.022142,-0.020041,0.032168,-0.010634,-0.031914,-0.011547,-0.022523
Scary Hours 2,-0.003230,-0.005584,-0.004388,-0.016800,-0.000730,0.020794,0.014782,-0.006227,-0.021592,-0.027542,...,0.010123,-0.008325,0.005029,-0.014950,-0.026921,0.043211,-0.013732,-0.032528,-0.006200,-0.037605
Scorpion,-0.010186,-0.015257,0.006393,-0.024187,-0.009992,0.016293,0.014764,-0.011984,-0.018830,-0.019814,...,0.019473,-0.005183,-0.001272,-0.021568,-0.024473,0.034897,-0.010928,-0.033972,-0.007297,-0.025549
So Far Gone,-0.009126,-0.020364,0.004072,-0.021837,-0.006309,0.016853,0.012548,-0.013841,-0.021235,-0.021233,...,0.022024,-0.007910,-0.004547,-0.026525,-0.020520,0.028277,-0.011030,-0.024399,-0.006087,-0.023669
Take Care,-0.004837,-0.020884,0.004196,-0.021686,-0.008491,0.011890,0.017436,-0.011215,-0.020619,-0.017806,...,0.020126,-0.003596,-0.003957,-0.024386,-0.025889,0.032859,-0.010794,-0.032833,-0.005136,-0.020893


#### Earth Mover's Distance
- Runs into the issue of flattening matrices

In [14]:
from scipy.stats import wasserstein_distance

# Flatten the matrices into 1D arrays
nwts_flattened = nwts_embeddings.flatten()
views_flattened = views_embeddings.flatten()

# Compute Earth Mover's Distance (Wasserstein distance)
emd_distance = wasserstein_distance(nwts_flattened, views_flattened)

print(f"Earth Mover's Distance between 'Nothing Was the Same' and 'Views': {emd_distance}")

Earth Mover's Distance between 'Nothing Was the Same' and 'Views': 0.00012117923816107082


#### Dynamic Time Warping

In [163]:
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
import numpy as np

#Create a dictionary of album names and their embeddings called album_embeddings from the drake_df dataframe
album_embeddings = {}

#remove rows in drake_df where the album is not 'Room for Improvement' or 'Comeback Season'
subset = drake_df[(drake_df['album'] != 'Room for Improvement') & (drake_df['album'] != 'Comeback Season')& (drake_df['album'] != 'Scary Hours 2')]

for album in subset['album'].unique():
    album_df = drake_df[drake_df['album'] == album]
    #Set the key as the album name and the value as stacked embeddings
    album_embeddings[album] = np.vstack(album_df['embedding'].values)

In [164]:
from scipy.spatial.distance import euclidean

# Assuming album_embeddings is a dictionary with album names as keys and numpy arrays as values
album_names = list(album_embeddings.keys())
n_albums = len(album_names)

# Initialize distance matrix
distance_matrix = np.zeros((n_albums, n_albums))

for i, name1 in enumerate(album_names):
    for j, name2 in enumerate(album_names):
        if i >= j:
            continue  # Skip redundant calculations
        dist, _ = fastdtw(album_embeddings[name1], album_embeddings[name2], dist=euclidean)
        distance_matrix[i, j] = dist
        distance_matrix[j, i] = dist  # Symmetric matrix

# Convert to DataFrame for easier handling
distance_df = pd.DataFrame(distance_matrix, columns=album_names, index=album_names)

In [165]:
from sklearn.manifold import MDS

# Initialize MDS
mds = MDS(n_components=2, dissimilarity='precomputed')

# Fit and transform
album_coordinates = mds.fit_transform(distance_matrix)

# Convert to DataFrame
album_coordinates_df = pd.DataFrame(album_coordinates, columns=['x', 'y'])
album_coordinates_df['album'] = album_names

/opt/anaconda3/lib/python3.8/site-packages/sklearn/manifold/_mds.py:298: FutureWarning:

The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.



### Create TSNE Visualizations

In [166]:
metadata = drake_df[['album', 'year', 'song_art_image_thumbnail_url']]
metadata = metadata.drop_duplicates(subset=['album'])

embedding_df = album_coordinates_df.merge(metadata, on='album', how='left')

In [167]:
# Assume embedding_df already exists with 'album', 'tsne-1', and 'tsne-2'

most_similar_albums = []
least_similar_albums = []

for i, row in embedding_df.iterrows():
    current_album = row['album']
    x1, y1 = row['x'], row['y']
    
    # Compute Euclidean distance for each other album
    distances = np.sqrt((embedding_df['x'] - x1)**2 + (embedding_df['y'] - y1)**2)
    
    # Remove the album itself from consideration
    distances = distances[embedding_df['album'] != current_album]
    
    # Identify most and least similar albums
    most_similar = distances.idxmin()
    least_similar = distances.idxmax()
    
    most_similar_albums.append(embedding_df.loc[most_similar, 'album'])
    least_similar_albums.append(embedding_df.loc[least_similar, 'album'])

# Add new columns to DataFrame
embedding_df['most_similar_album'] = most_similar_albums
embedding_df['least_similar_album'] = least_similar_albums

In [168]:
import plotly.io as pio
import plotly.express as px
pio.templates.default = "plotly_dark"

# Create a square figure
fig = px.scatter(embedding_df, x='x', y='y', opacity=0, hover_data=['year', 'most_similar_album', 'least_similar_album'])

# Calculate the range based on your actual data for a dynamic approach
x_min = embedding_df['x'].min() - 5  # 10 units padding
x_max = embedding_df['x'].max() + 5
y_min = embedding_df['y'].min() - 10
y_max = embedding_df['y'].max() + 0

# Update x-axis and y-axis range
fig.update_xaxes(range=[x_min, x_max])
fig.update_yaxes(range=[y_min, y_max])

# Remove axis labels and numbers
fig.update_xaxes(title_text='', showticklabels=False)
fig.update_yaxes(title_text='', showticklabels=False)

# Make plot square
fig.update_layout(width=1000, height=1000, title_text='Lyrical Similarity Between Drake Albums', title_x=0.5)

fig.update_layout(title_font=dict(size=24, family="Arial, sans-serif"))

# Add images
for i, row in embedding_df.iterrows():
    fig.add_layout_image(
        dict(
            source=row['song_art_image_thumbnail_url'],
            x=row['x'],
            y=row['y'],
            xref="x",
            yref="y",
            xanchor="center",
            yanchor="top",
            sizex=3,  # you may need to adjust these dimensions based on your specific case
            sizey=3,
            opacity=1,
            layer="below"
        )
    )

    # Add text label below the image
    fig.add_annotation(
        x=row['x'],
        y=row['y'],
        xanchor="center",
        yanchor="bottom",
        text=row['album'],
        showarrow=False,
        font=dict(size=10)
    )

#Remove gridlines
#fig.update_xaxes(showgrid=False, zeroline=False)
#fig.update_yaxes(showgrid=False, zeroline=False)

#fig.update_layout(paper_bgcolor="rgba(0,0,0,0)", plot_bgcolor="rgba(0,0,0,0)")
#fig.update_layout(paper_bgcolor='#fafafa', plot_bgcolor='#fafafa')

# Show the plot
fig.show()

### Visualize Every Drake Song

In [185]:
# Convert the embeddings column of lists to a 2D numpy array
embeddings = drake_df['embedding'].tolist()
embeddings_array = np.array(embeddings)

# Apply t-SNE transformation
tsne = TSNE(n_components=2, perplexity=50, n_iter=300, random_state=2)
embeddings_2d = tsne.fit_transform(embeddings_array)

song_embeddings_df = pd.DataFrame(data=embeddings_2d, columns=['Dimension 1', 'Dimension 2'])
song_embeddings_df['title'] = drake_df['title']
song_embeddings_df['album'] = drake_df['album']
#Remove the rows where the album is 'Room for Improvement' or 'Comeback Season'
song_embeddings_df = song_embeddings_df.loc[~song_embeddings_df['album'].isin(['Room for Improvement', 'Comeback Season'])]
song_embeddings_df['year'] = drake_df['year']
song_embeddings_df['song_art_image_thumbnail_url'] = drake_df['song_art_image_thumbnail_url']
#Remove rows where the title ares duplicates, and only keep the row with the highest year
song_embeddings_df = song_embeddings_df.sort_values('year', ascending=False).drop_duplicates(subset=['title'])
#Remove the row where the title is 'Hate Sleeping Alone'
song_embeddings_df = song_embeddings_df[song_embeddings_df['title'] != 'Hate Sleeping Alone']
song_embeddings_df = song_embeddings_df[song_embeddings_df['title'] != 'More Life: Editor’s Note']
song_embeddings_df.head()

,Dimension 1,Dimension 2,title,album,year,song_art_image_thumbnail_url
54,-2.590371,-1.825695,Calling For You,For All The Dogs,2023,https://images.genius.com/2974a46656bda32e9b44...
52,4.111679,0.787449,Virginia Beach,For All The Dogs,2023,https://images.genius.com/2974a46656bda32e9b44...
55,-4.190627,-3.310348,Fear Of Heights,For All The Dogs,2023,https://images.genius.com/2974a46656bda32e9b44...
56,-0.310613,-5.651745,Daylight,For All The Dogs,2023,https://images.genius.com/2974a46656bda32e9b44...
57,-1.683655,-3.729206,First Person Shooter,For All The Dogs,2023,https://images.genius.com/2974a46656bda32e9b44...


In [207]:
from plotly import express as px

# Step 1: Sort albums by release year and create a color map
unique_albums = song_embeddings_df['album'].unique()
album_years = {album: song_embeddings_df[song_embeddings_df['album']==album]['year'].iloc[0] for album in unique_albums}
sorted_albums = sorted(album_years.keys(), key=lambda x: album_years[x])

colors = px.colors.qualitative.Plotly  # You can choose a color palette that fits your needs
album_color_map = {album: colors[i % len(colors)] for i, album in enumerate(sorted_albums)}

# Step 2: Map the color assignments back to the DataFrame
song_embeddings_df['color'] = song_embeddings_df['album'].map(album_color_map)

# Initialize the scatter plot
fig = px.scatter(
    song_embeddings_df,
    x='Dimension 1',
    y='Dimension 2',
    text='title',
    color='album',
    color_discrete_sequence=[album_color_map[album] for album in sorted_albums],
    hover_data=['album', 'year'],
)

# Calculate the axis ranges dynamically
x_min = song_embeddings_df['Dimension 1'].min() - 2
x_max = song_embeddings_df['Dimension 1'].max() + 2
y_min = song_embeddings_df['Dimension 2'].min() - 0.5
y_max = song_embeddings_df['Dimension 2'].max() + 0.5

# Apply all updates to the plot
fig.update_layout(
    width=1200,
    height=1000,
    title_text='Lyrical Similarity Between Drake Songs',
    title_x=0.5,
    xaxis=dict(range=[x_min, x_max], showticklabels=False, title=''),
    yaxis=dict(range=[y_min, y_max], showticklabels=False, title=''),
    legend=dict(
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=9,  # Text size
            color="white"  # Text color
        )
    )
)


# Update trace properties
fig.update_traces(textposition='top center', textfont=dict(color="white", size=9))


# Display the updated plot
fig.show()


## For all the dogs

In [113]:
fatd = drake_df.loc[drake_df['album'] == 'For All The Dogs'].reset_index(drop=True)
# Convert the embeddings column of lists to a 2D numpy array
embeddings = fatd['embedding'].tolist()
embeddings_array = np.array(embeddings)

# Apply t-SNE transformation
tsne = TSNE(n_components=2, perplexity=10, n_iter=300, random_state=42)
embeddings_2d = tsne.fit_transform(embeddings_array)

embeddings_df = pd.DataFrame(data=embeddings_2d, columns=['Dimension 1', 'Dimension 2'])
#Join the 'title' column from fatd to the embeddings_df dataframe
embeddings_df['title'] = fatd['title']
embeddings_df.head()

,Dimension 1,Dimension 2,title
0,-9.782958,18.513693,Virginia Beach
1,24.111506,30.840143,Amen
2,29.857950,-0.703904,Calling For You
3,18.330439,-14.833846,Fear Of Heights
4,7.720140,3.040021,Daylight


In [130]:
#Show the scatter plot
fig = px.scatter(embeddings_df, x='Dimension 2', y='Dimension 1', text='title')

# Calculate the axis ranges dynamically
x_min = embeddings_df['Dimension 2'].min() - 7
x_max = embeddings_df['Dimension 2'].max() + 7
y_min = embeddings_df['Dimension 1'].min() - 0.5
y_max = embeddings_df['Dimension 1'].max() + 3

# Remove axis labels and numbers
fig.update_xaxes(title_text='', showticklabels=False)
fig.update_yaxes(title_text='', showticklabels=False)

# Move text labels above the dots
fig.update_traces(textposition='top center')

# Add a title
fig.update_layout(title_text='Lyrical Similarity for Songs on "For All The Dogs"',
                  width=800,
                  height=800,
                  title_x=0.5,
                  xaxis=dict(range=[x_min, x_max]),
                  yaxis=dict(range=[y_min, y_max])

)
#Add a background picture to the plot
fig.add_layout_image(
    dict(
        source = fatd['song_art_image_thumbnail_url'][0],
              xref="paper",
        yref="paper",
        x=0,
        y=1,
        sizex=1,
        sizey=1,
        xanchor="left",
        yanchor="top",
        opacity=0.5,
        layer="below"
    )
)

# Show the plot
fig.show()